<a href="https://colab.research.google.com/github/zoebatz/CAP6671_project/blob/main/finetuning_on_DRD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

finetuning BERT and MentalBERT on DRD


*   split DRD 80/10/10 for training, validation and testing
*   



In [ ]:
''' !pip install transformers torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.9 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import torch
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from evaluate import load
import numpy as np

In [ ]:
'''!pip install wandb

In [ ]:
import wandb
import os

from google.colab import userdata
os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: zoe-batz99 (zoe-batz99-university-of-central-florida) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# load dataset to use for fine tuning
file_name = 'data_DRD'

path = '/content/drive/MyDrive/Colab Notebooks/project_final/datasets/'+file_name+'.csv'
data = pd.read_csv(path)
data.head()

,text,label
0,struggling hard with inventory,0
1,sad that the foot of my macbook just fell off,0
2,i m a high school student f and i m not living...,1
3,flowersophy i slept so much when i wa a little...,0
4,is going to priceline city tomorrow but lost h...,0


In [ ]:
len(data)

7731

In [ ]:
data['label'].value_counts()

,count
label,
0,3900
1,3831


In [ ]:

model_name = "bert-base-uncased" # basic model
# "mental/mental-bert-base-uncased" # model fine tuned on mental health data
# "bert-base-uncased" # basic model

# tokenize data
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# convert to hugging face dataset
data = Dataset.from_pandas(data)
tokenized_data = data.map(tokenize_function, batched=True)
tokenized_data.set_format("torch")

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/7731 [00:00<?, ? examples/s]

In [ ]:
# remove unnecessary columns (untokenized data)
tokenized_data = tokenized_data.remove_columns(['text'])

In [ ]:
# split data into train, val and test 80/10/10
train_data, test_data = tokenized_data.train_test_split(test_size=0.2, shuffle=True, seed=42).values()
eval_data, test_data = test_data.train_test_split(test_size=0.5, shuffle=True, seed=42).values()


In [ ]:
train_data

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6184
})

In [ ]:
eval_data

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 773
})

In [ ]:
test_data

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 774
})

In [ ]:
# load pretrained base model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# define metrics
import numpy as np
from evaluate import load

metric_acc = load("accuracy")
metric_prec = load("precision")
metric_rec = load("recall")
metric_f1 = load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    acc = metric_acc.compute(predictions=predictions, references=labels)
    prec = metric_prec.compute(predictions=predictions, references=labels, average="binary")
    rec = metric_rec.compute(predictions=predictions, references=labels, average="binary")
    f1 = metric_f1.compute(predictions=predictions, references=labels, average="binary")
    return {"accuracy": acc["accuracy"], "precision": prec["precision"], "recall": rec["recall"], "f1": f1["f1"]}



In [ ]:
# adjust parameters
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    eval_strategy="epoch",  # eval after every epoch
    save_strategy="epoch",  # save model after every epoch
    learning_rate=3e-5, # try varying 1, 2, 3, 4, 5
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=200,
    weight_decay=0.01,  # regularization
    logging_dir="./logs",
    logging_steps=50,    # log every 100 steps
    load_best_model_at_end=True,
    metric_for_best_model="f1", # better for imbalanced classes
    greater_is_better=True,   # higher f1 is better
    report_to="wandb"  # Enables logging to Weights & Biases
)

In [ ]:
# train model

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

trainer.train()
#trainer.state.log_history

<ipython-input-19-ce7f4f2e13b3>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.107200,0.066071,0.978008,0.997183,0.956757,0.976552
2,0.030800,0.055797,0.988357,0.989160,0.986486,0.987821
3,0.000500,0.057836,0.990944,0.989218,0.991892,0.990553


TrainOutput(global_step=1161, training_loss=0.06747380227594926, metrics={'train_runtime': 392.9992, 'train_samples_per_second': 47.206, 'train_steps_per_second': 2.954, 'total_flos': 4881236299038720.0, 'train_loss': 0.06747380227594926, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.05783601105213165,
 'eval_accuracy': 0.9909443725743855,
 'eval_precision': 0.9892183288409704,
 'eval_recall': 0.9918918918918919,
 'eval_f1': 0.99055330634278,
 'eval_runtime': 4.7677,
 'eval_samples_per_second': 162.133,
 'eval_steps_per_second': 5.244,
 'epoch': 3.0}

In [ ]:
results = trainer.evaluate(test_data)
print(results)

{'eval_loss': 0.10093621164560318, 'eval_accuracy': 0.9819121447028424, 'eval_precision': 0.9815789473684211, 'eval_recall': 0.9815789473684211, 'eval_f1': 0.9815789473684211, 'eval_runtime': 4.803, 'eval_samples_per_second': 161.148, 'eval_steps_per_second': 5.205, 'epoch': 3.0}


In [ ]:
# save fine tuned model to drive
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/fine_tuned_mental_bert')
tokenizer.save_pretrained('/content/drive/MyDrive/Colab Notebooks/fine_tuned_mental_bert')

('/content/drive/MyDrive/Colab Notebooks/fine_tuned_mental_bert/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/fine_tuned_mental_bert/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/fine_tuned_mental_bert/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/fine_tuned_mental_bert/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/fine_tuned_mental_bert/tokenizer.json')